# Object Models

 * `OrbitState`: initial orbit state with OMM-based attributes as a component of the propagation analysis input
 * `PropagationRequest`: input to propagation analysis function
 * `PropagationRecord`: time, position, and velocity attributes as a component of the propagation analysis output
 * `PropagationResponse`: output from propagation analysis function

In [25]:
from datetime import datetime, timedelta
from enum import Enum
from typing import Annotated, List, Optional, Union

from pydantic import AwareDatetime, BaseModel, Field
from sgp4 import exporter, omm
from sgp4.api import Satrec


class OrbitState(BaseModel):
    object_name: Optional[str] = Field(None, description="Object name.")
    object_id: Optional[str] = Field(None, description="Object identifier.")
    epoch: datetime = Field(
        ..., description="Epoch (time of orbit element specification)."
    )
    mean_motion: float = Field(
        ..., gt=0, description="Mean motion (revolutions per day)."
    )
    eccentricity: float = Field(..., ge=0, lt=1, description="Eccentricity.")
    inclination: float = Field(..., ge=0, le=180, description="Inclination (degrees).")
    ra_of_asc_node: float = Field(
        ..., ge=0, lt=360, description="Right ascension of ascending node (degrees)."
    )
    arg_of_pericenter: float = Field(
        ..., ge=0, lt=360, description="Argument of pericenter (degrees)."
    )
    mean_anomaly: float = Field(
        ..., ge=0, lt=360, description="Mean anomaly (degrees)."
    )
    ephemeris_type: Optional[int] = Field(
        0,
        ge=0,
        description="Ephemeris type (0: Default (Kozai), 1: SGP, 2: SGP4 (Brouwer), 3: SDP4, 4: SGP8, 5: SDP8).",
    )
    classification_type: Optional[str] = Field(
        "U",
        pattern="[UCS]",
        description="Classification (U: unclassified, C: classified, S: secret).",
    )
    norad_cat_id: Optional[int] = Field(
        None, ge=0, description="NORAD catalog identifier."
    )
    element_set_no: Optional[int] = Field(
        None, ge=0, le=999, description="Element set number."
    )
    rev_at_epoch: Optional[int] = Field(
        None, ge=0, le=99999, description="Revolution number at epoch."
    )
    bstar: float = Field(
        0.0,
        ge=0,
        lt=1,
        description="B-star drag term (radiation pressure coefficient).",
    )
    mean_motion_dot: float = Field(
        0.0,
        gt=-1,
        lt=1,
        description="First derivative of mean motion (ballistic coefficient).",
    )
    mean_motion_ddot: float = Field(
        0.0, gt=-1, lt=1, description="Second derivative of mean motion."
    )

    @classmethod
    def from_omm(cls, omm) -> "OrbitState":
        return OrbitState.model_validate(
            dict([(key.lower(), value) for key, value in omm.items()])
        )

    def to_omm(self) -> dict:
        return dict(
            [
                (
                    key.upper(),
                    (
                        ""
                        if value is None
                        else value.isoformat() if isinstance(value, datetime) else value
                    ),
                )
                for key, value in self.model_dump().items()
            ]
        )

    def to_satrec(self) -> Satrec:
        sat = Satrec()
        omm.initialize(sat, self.to_omm())
        return sat

    def to_tle(self) -> List[str]:
        return exporter.export_tle(self.to_satrec())


class ReferenceFrame(str, Enum):
    ECEF = "ECEF"  # Earth-Centered Earth-Fixed Reference Frame
    ICRF = "ICRF"  # International Celestial Reference Frame


class PropagationRequest(BaseModel):
    orbit: OrbitState = Field(..., description="Orbit to be propagated.")
    start: AwareDatetime = Field(..., description="Propagation start time.")
    duration: timedelta = Field(..., description="Propagation duration.")
    time_step: timedelta = Field(..., description="Propagation time step duration.")
    frame: Union[ReferenceFrame, str] = Field(
        ReferenceFrame.ICRF,
        description="Reference frame in which propagation results defined.",
    )


Vector = Annotated[
    List[float],
    Field(min_length=3, ma_length=3, description="Cartesian vector (x,y,z)."),
]

Quaternion = Annotated[
    List[float],
    Field(min_length=4, max_length=4, description="Quaternion (x,y,z,w)."),
]


class FixedOrientation(str, Enum):
    NADIR_GEOCENTRIC = "GEOCENTRIC"  # pointing through geocenter
    NADIR_GEODETIC = "GEODETIC"  # normal to ellipsoid surface


class PropagationRecord(BaseModel):
    time: AwareDatetime = Field(..., description="Time")
    position: Vector = Field(
        ...,
        description="Position (m)",
    )
    velocity: Vector = Field(
        ...,
        description="Velocity (m/s)",
    )
    body_orientation: Optional[Union[FixedOrientation, Quaternion]] = Field(
        None,
        description="Orientation of the spacecraft body-fixed frame, relative to requested frame.",
    )
    view_orientation: Optional[Quaternion] = Field(
        [0, 0, 0, 1],
        description="Orientation of the instrument view, relative to the body-fixed frame.",
    )


class PropagationResponse(BaseModel):
    records: List[PropagationRecord] = Field([], description="Propagation results")
    frame: Union[ReferenceFrame, str] = Field(
        ReferenceFrame.ICRF,
        description="Reference frame in which position/velocity defined.",
    )

# TAT-C Interface Implementation

Specifies a function with well-defined input (`PropagationRequest`) and output (`PropagationResponse`) using TAT-C's implementation.

In [26]:
from tatc.schemas import Instrument, Satellite, TwoLineElements
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput

import pandas as pd


def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            Satellite(
                name=request.orbit.object_name,
                orbit=TwoLineElements(tle=request.orbit.to_tle()),
            ),
            Instrument(name="Instrument"),
            pd.date_range(
                request.start, request.start + request.duration, freq=request.time_step
            ),
            coordinates=(
                OrbitCoordinate.ECI
                if request.frame == ReferenceFrame.ICRF
                else (
                    OrbitCoordinate.ECEF
                    if request.frame == ReferenceFrame.ECEF
                    else RuntimeError(f"{request.frame} not implemented.")
                )
            ),
            orbit_output=OrbitOutput.POSITION_VELOCITY,
        ).apply(
            lambda r: PropagationRecord(
                time=r.time,
                position=r.geometry.coords[0],
                velocity=r.velocity.coords[0],
                body_orientation=FixedOrientation.NADIR_GEOCENTRIC,
            ),
            axis=1,
        ),
        frame=request.frame,
    )

# Example

Pulls OMM file from Celestrak, issues propagation request starting Jan 1 2024 for 1 day (1 minute time step), and displays results.

Note that results are formatted as Python objects. The JSON-serialized outputs can be produced by replacing `model_dump` with `model_dump_json`; however, they do not display nicely in a Jupyter notebook.

In [27]:
import requests
import json
from datetime import datetime, timedelta, timezone

# note: celestrak request is rate-limited; using hard-coded version
# response = requests.get("https://celestrak.org/NORAD/elements/gp.php?NAME=ZARYA&FORMAT=JSON").content
response = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss = json.loads(response)[0]

In [28]:
request = PropagationRequest(
    orbit=OrbitState.from_omm(iss),
    start=datetime(2024, 1, 1, tzinfo=timezone.utc),
    duration=timedelta(days=1),
    time_step=timedelta(minutes=1),
)

display(request.model_dump())

response = propagate_tatc(request)

display(response.model_dump())

{'orbit': {'object_name': 'ISS (ZARYA)',
  'object_id': '1998-067A',
  'epoch': datetime.datetime(2024, 6, 7, 9, 53, 34, 728000),
  'mean_motion': 15.50975122,
  'eccentricity': 0.0005669,
  'inclination': 51.6419,
  'ra_of_asc_node': 3.7199,
  'arg_of_pericenter': 284.672,
  'mean_anomaly': 139.0837,
  'ephemeris_type': 0,
  'classification_type': 'U',
  'norad_cat_id': 25544,
  'element_set_no': 999,
  'rev_at_epoch': 45703,
  'bstar': 0.00033759,
  'mean_motion_dot': 0.00019541,
  'mean_motion_ddot': 0.0},
 'start': datetime.datetime(2024, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'duration': datetime.timedelta(days=1),
 'time_step': datetime.timedelta(seconds=60),
 'frame': <ReferenceFrame.ICRF: 'ICRF'>}

{'records': [{'time': Timestamp('2024-01-01 00:00:00+0000', tz='UTC'),
   'position': [-4499686.1202688, -232613.21550674032, 5091657.311614182],
   'velocity': [-1677.6634157866658, -7252.659671818871, -1804.8077999287968],
   'body_orientation': <FixedOrientation.NADIR_GEOCENTRIC: 'GEOCENTRIC'>,
   'view_orientation': [0, 0, 0, 1]},
  {'time': Timestamp('2024-01-01 00:01:00+0000', tz='UTC'),
   'position': [-4590028.087700935, -666915.9700864203, 4971828.475405545],
   'velocity': [-1332.576798616102, -7218.509492576277, -2187.9548996394055],
   'body_orientation': <FixedOrientation.NADIR_GEOCENTRIC: 'GEOCENTRIC'>,
   'view_orientation': [0, 0, 0, 1]},
  {'time': Timestamp('2024-01-01 00:02:00+0000', tz='UTC'),
   'position': [-4659474.854577173, -1098182.798715333, 4829301.789743175],
   'velocity': [-981.4248032269486, -7151.498310442072, -2561.1120354711065],
   'body_orientation': <FixedOrientation.NADIR_GEOCENTRIC: 'GEOCENTRIC'>,
   'view_orientation': [0, 0, 0, 1]},
  {'time': 